# Spotify x Billboard Five Year Analysis Project: Exploratory Analysis

## Import packages 

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Set Display Options

In [2]:
pd.set_option('display.max_columns', 500)

## Import dataframes

### Import All Chart Tracks Table

In [3]:
df1=pd.read_csv("../data/AllChartAlbumTracksRecoded.csv")

In [4]:
df1 = df1.drop("Unnamed: 0", axis=1)
df1

,track_name,track_id,track_popularity,track_artists,track_artists_ids,track_artists_genres,track_artists_popularity,track_explicit,track_artists_followers,album_name,album_id,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,track_artist,track_artist_id,track_artist_popularity,track_artist_followers,track_artists_genres_consolidate,Misc,Rock,Pop,Hip-hop,Edm,Reggaeton,Indie,Christian,Country,Soul,Metal,Jazz,Folk,Latin,Funk,Easy_listening,R&B,Reggae,Show_tunes
0,Sand In My Boots,4qeMbs55QlONyrE9YBMA93,42,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.354,0.540,3.0,-6.625,1.0,0.587,0.000000,0.1120,0.416,69.970,202133.0,4.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,Wasted On You,4xv9BDq764NSKG3geku9X2,43,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.505,0.658,11.0,-5.240,0.0,0.371,0.001360,0.1210,0.255,196.002,178520.0,3.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Somebody’s Problem,7cWMnSxmQfKFsIIoRWifbb,36,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.658,0.539,5.0,-7.674,1.0,0.665,0.000000,0.1230,0.622,136.959,161773.0,4.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,More Surprised Than Me,6Mn2GFiNNadK0G2ZXRK1fd,36,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.488,0.748,6.0,-6.049,1.0,0.244,0.000000,0.1240,0.431,84.938,157440.0,4.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,865,3lb2f27695Iuyd82khwU9k,39,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.454,0.586,8.0,-5.254,1.0,0.729,0.000000,0.1010,0.366,87.891,190680.0,4.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8991,Home,32zF5eXrb8QrUu5qNdYmIO,36,['Lady A'],['32WkQRZEVKSzVAAYqukAEA'],"['contemporary country', 'country', 'country d...",[74],False,[3424979],Heart Break,4WZZDzGqjvw4bRwrp56U24,0.559,0.692,7.0,-7.797,1.0,0.486,0.000004,0.1250,0.567,89.954,177200.0,4.0,Lady A,32WkQRZEVKSzVAAYqukAEA,74,3424979,"['Country', 'Country', 'Country', 'Pop', 'Coun...",0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8992,Famous,5xeo7Mr6638HYaEontltHV,35,['Lady A'],['32WkQRZEVKSzVAAYqukAEA'],"['contemporary country', 'country', 'country d...",[74],False,[3424979],Heart Break,4WZZDzGqjvw4bRwrp56U24,0.557,0.621,0.0,-6.355,0.0,0.597,0.000098,0.0864,0.680,76.994,232613.0,4.0,Lady A,32WkQRZEVKSzVAAYqukAEA,74,3424979,"['Country', 'Country', 'Country', 'Pop', 'Coun...",0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8993,Coming Clean,3p5JBbqJpfHqPpSdkjZbfX,12,['Lil Durk'],['3hcs9uc56yIGFCSy9leWe7'],"['chicago drill', 'chicago rap', 'drill', 'hip...",[89],False,[3405916],The Voice,1Gonl2Jwl4WeJM6qUomM8s,0.763,0.471,8.0,-10.458,1.0,0.283,0.000000,0.1300,0.525,140.057,147413.0,4.0,Lil Durk,3hcs9uc56yIGFCSy9leWe7,89,3405916,"['Hip-hop', 'Hip-hop', 'Hip-hop', 'Hip-hop', '...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8994,Perfecto,0Xcy81PsJCOO6mhLZaQyQ4,66,['Mac Miller'],['4LLpKhyESsyAXpc4laK94U'],"['hip hop', 'pittsburgh rap', 'rap']",[87],True,[6788769],Swimming,5wtE5aLX5r7jOosmPhJhhk,0.599,0.317,1.0,-11.212,1.0,0.748,0.000000,0.1110,0.203,148.090,215413.0,4.0,Mac Miller,4LLpKhyESsyAXpc4laK94U,87,6788769,"['Hip-ho

### Import Master Chart Table

In [5]:
df2=pd.read_csv("../data/MasterChartTable.csv")
df2

,Unnamed: 0,playlist_id,playlist_name,position,album_name,album_id,album_release_date,album_artists,album_artists_ids,album_artists_genres,album_artists_popularity,album_artists_followers,track_name,track_id,track_popularity,track_artists,track_artists_ids,track_artists_genres,track_artists_popularity,track_explicit,track_artists_followers,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,2021-01-08,Morgan Wallen,['4oUHIQIBe0LHzYfvXNW4QM'],[['contemporary country']],[86],[2705213],More Than My Hometown,65mMCEOu5Ll1DBAfEUmerU,34,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],[['contemporary country']],[86],False,[2705213],0.621,0.868,6,-5.478,1,0.617000,0.000000,0.1310,0.594,126.010,216573,4
1,1,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,2,SOUR,6s84u2TUpR3wdUv4NgKA2j,2021-05-21,['Olivia Rodrigo'],['1McMsnEElThX1knmY4oliG'],[['pop']],[92],[12088661],drivers license,5wANPM4fQCJwkGd4rN57mH,92,['Olivia Rodrigo'],['1McMsnEElThX1knmY4oliG'],[['pop']],[92],True,[12088661],0.561,0.431,10,-8.810,1,0.768000,0.000014,0.1060,0.137,143.875,242013,4
2,2,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,3,Shoot For The Stars Aim For The Moon,7e7t0MCrNDcJZsPwUKjmOc,2020-07-03,['Pop Smoke'],['0eDvMgVFoNV3TpwtrVCoTj'],[['brooklyn drill']],[89],[8721767],What You Know Bout Love,1tkg4EHVoqnhR6iFEXb60y,85,['Pop Smoke'],['0eDvMgVFoNV3TpwtrVCoTj'],[['brooklyn drill']],[89],True,[8721767],0.709,0.548,10,-8.493,1,0.650000,0.000002,0.1330,0.543,83.995,160000,4
3,3,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,4,evermore,2Xoteh7uEpea4TohMxjtaq,2020-12-11,['Taylor Swift'],['06HL4z0CvFAxyc27GXpf02'],[['pop']],[100],[47362363],willow,0lx2cLdOt3piJbcaXIV74f,82,['Taylor Swift'],['06HL4z0CvFAxyc27GXpf02'],[['pop']],[100],False,[47362363],0.392,0.574,7,-9.195,1,0.833000,0.001790,0.1450,0.529,81.112,214707,4
4,4,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,5,Certified Lover Boy,3SpBlxme9WbeQdI9kx7KAV,2021-09-03,['Drake'],['3TVXtAsR1Inumwj472S9r4'],"[['canadian hip hop', 'canadian pop', 'hip hop...",[98],[59845429],Way 2 Sexy (with Future & Young Thug),0k1WUmIRnG3xU6fvvDVfRG,90,"['Drake', 'Future', 'Young Thug']",['3TVXtAsR1Inumwj472S9r4'],"[['canadian hip hop', 'canadian pop', 'hip hop...",[98],True,[59845429],0.803,0.597,11,-6.035,0,0.000619,0.000005,0.3230,0.331,136.008,257605,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,988,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,195,+,0W5GGnapMz0VwemQvJDqa7,2011-09-09,['Ed Sheeran'],['6eUKZXaKkcviH0Ku9w2n3V'],"[['pop', 'uk pop']]",[97],[90231430],The A Team,1VdZ0vKfR5jneCmWIUAMxK,78,['Ed Sheeran'],['6eUKZXaKkcviH0Ku9w2n3V'],"[['pop', 'uk pop']]",[97],False,[90231430],0.642,0.289,9,-9.918,1,0.669000,0.000000,0.1800,0.407,84.996,258373,4
989,989,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,196,Sing It Now: Songs Of Faith & Hope,3zSyGvLk5FcLhC3BtbKr9z,2017-02-03,['Reba McEntire'],['02rd0anEWfMtF7iMku9uor'],"[['contemporary country', 'country', 'country ...",[67],[1535356],Back To God,3W29VxuZ2AZNkPPJwVmD3a,34,['Reba McEntire'],['02rd0anEWfMtF7iMku9uor'],"[['contemporary country', 'country', 'country ...",[67],False,[1535356],0.501,0.674,6,-5.666,1,0.039000,0.000003,0.0783,0.359,147.931,289840,4
990,990,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,197,BEYONCÉ [Platinum Edition],2UJwKSBUz6rtW4QLK74kQu,2014-11-24,['Beyoncé'],['6vWDO969PvNqNYHIOW5v0m'],"[['dance pop', 'pop', 'r&b']]",[87],[29904174],7/11,02M6vucOvmRfMxTXDUwRXu,73,['Beyoncé'],['6vWDO969PvNqNYHIOW5v0m'],"[['dance pop', 'pop', 'r&b']]",[87],False,[29904174],0.747,0.705,9,-5.137,0,0.012800,0.000000,0.1260,0.560,136.024,213507,4
991,991,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,199,The Breaker,2aQOzEjLzPkffXDwREXdAh,2017-02-24,['Little

In [6]:
df3 = df2[["playlist_id", "playlist_name", "position", "album_release_date", "album_id"]]
df3

,playlist_id,playlist_name,position,album_release_date,album_id
0,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08,1qW1C4kDOXnrly22daHbxz
1,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,2,2021-05-21,6s84u2TUpR3wdUv4NgKA2j
2,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,3,2020-07-03,7e7t0MCrNDcJZsPwUKjmOc
3,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,4,2020-12-11,2Xoteh7uEpea4TohMxjtaq
4,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,5,2021-09-03,3SpBlxme9WbeQdI9kx7KAV
...,...,...,...,...,...
988,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,195,2011-09-09,0W5GGnapMz0VwemQvJDqa7
989,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,196,2017-02-03,3zSyGvLk5FcLhC3BtbKr9z
990,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,197,2014-11-24,2UJwKSBUz6rtW4QLK74kQu
991,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,199,2017-02-24,2aQOzEjLzPkffXDwREXdAh


### Join Master Chart and All Track Tables on the Album ID

In [7]:
df4 = pd.merge(df1, df3, on='album_id')

In [8]:
df4

,track_name,track_id,track_popularity,track_artists,track_artists_ids,track_artists_genres,track_artists_popularity,track_explicit,track_artists_followers,album_name,album_id,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,track_artist,track_artist_id,track_artist_popularity,track_artist_followers,track_artists_genres_consolidate,Misc,Rock,Pop,Hip-hop,Edm,Reggaeton,Indie,Christian,Country,Soul,Metal,Jazz,Folk,Latin,Funk,Easy_listening,R&B,Reggae,Show_tunes,playlist_id,playlist_name,position,album_release_date
0,Sand In My Boots,4qeMbs55QlONyrE9YBMA93,42,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.354,0.540,3.0,-6.625,1.0,0.58700,0.000000,0.1120,0.416,69.970,202133.0,4.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08
1,Wasted On You,4xv9BDq764NSKG3geku9X2,43,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.505,0.658,11.0,-5.240,0.0,0.37100,0.001360,0.1210,0.255,196.002,178520.0,3.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08
2,Somebody’s Problem,7cWMnSxmQfKFsIIoRWifbb,36,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.658,0.539,5.0,-7.674,1.0,0.66500,0.000000,0.1230,0.622,136.959,161773.0,4.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08
3,More Surprised Than Me,6Mn2GFiNNadK0G2ZXRK1fd,36,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.488,0.748,6.0,-6.049,1.0,0.24400,0.000000,0.1240,0.431,84.938,157440.0,4.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08
4,865,3lb2f27695Iuyd82khwU9k,39,['Morgan Wallen'],['4oUHIQIBe0LHzYfvXNW4QM'],['contemporary country'],[87],False,[2705213],Dangerous: The Double Album,1qW1C4kDOXnrly22daHbxz,0.454,0.586,8.0,-5.254,1.0,0.72900,0.000000,0.1010,0.366,87.891,190680.0,4.0,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,87,2705213,['Country'],0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16500,Big Love In A Small Town,1VwuDXLQdyTWvieBIPWkNt,37,['Lady A'],['32WkQRZEVKSzVAAYqukAEA'],"['contemporary country', 'country', 'country d...",[74],False,[3424979],Heart Break,4WZZDzGqjvw4bRwrp56U24,0.524,0.523,1.0,-5.150,1.0,0.64500,0.000000,0.1010,0.313,144.012,226013.0,4.0,Lady A,32WkQRZEVKSzVAAYqukAEA,74,3424979,"['Country', 'Country', 'Country', 'Pop', 'Coun...",0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,200,2017-06-09
16501,The Stars,2gFkkpMs7rakuCCi9UepP0,37,['Lady A'],['32WkQRZEVKSzVAAYqukAEA'],"['contemporary country', 'country', 'country d...",[74],False,[3424979],Heart Break,4WZZDzGqjvw4bRwrp56U24,0.576,0.766,1.0,-4.594,1.0,0.26300,0.000004,0.1020,0.407,143.029,202347.0,4.0,Lady A,32WkQRZEVKSzVAAYqukAEA,74,3424979,"['Country', 'Country', 'Country', 'Pop', 'Coun...",0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1N9WQ0C6m7n1TVeLXdUoNb,Billboard 200 Top Albums 2017,200,2017-06-09
16502,Teenage Heart,7G2BnhS46nEiPucLNvE0G3,36,['Lady A'],['32WkQRZEVKSzVAAYqukAEA'],"['contemporary country', 'country', 'count

## Explorations

### Preliminary Questions

#### Shape of the dataframe:

In [27]:
print(f"\nDataframe shape: {df4.shape}\n") 
# 48 columns columns (data points), 8996 rows (songs)


Dataframe shape: (16505, 51)



In [26]:
print(f"Most popular song: {df4[df4['track_popularity'] == 100]['track_name'].item()}")

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
print(f"Most popular song: {df[df['popularity'] == 100]['name'].item()}"
        f" by {df[df['popularity'] == 100]['artists'].item()}\n")
print(f"Average loudness for all years: {np.mean(df['loudness'])} dB\n")
print(f"Average loudness in 2020: {np.mean(df[df['year'] == 2020]['loudness'])} dB\n")
print(f"Longest song: {df.iloc[df['duration_ms'].idxmax()]['name']} by "
        f"{df.iloc[df['duration_ms'].idxmax()]['artists']}, length = "
        f"{df['duration_ms'].max() / 60000} minutes\n")
print("Number of songs with Drake: " + str(df['artists'].str.contains("'Aso'").sum()) + "\n")
print("Number of songs with Kanye West and Drake: " + str(df['artists'].str.contains(
        r".*'Drake'.*'Kanye West'.*|.*'Kanye West'.*'Drake'.*").sum()) + "\n")
print("Most popular Kanye West song: " + str(df.iloc[df[df['artists'].str.contains("'Kanye West'")]['popularity'].idxmax]['name']) + "\n")
